# Sample 5: LangChain with AI Foundry

This notebook demonstrates using LangChain to interact with AI Foundry models. LangChain provides a higher-level abstraction over the OpenAI API with additional features like prompt templates, chains, and memory.

## Key Features
- LangChain ChatOpenAI integration
- Prompt templates
- Conversation memory
- Chain operations

## 1. Environment Setup

In [30]:
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv('../../.env')

# Verify required environment variables
required_vars = [
    'AZURE_OPENAI_ENDPOINT',
    'AZURE_OPENAI_API_KEY', 
    'AZURE_OPENAI_DEPLOYMENT_NAME'
]

for var in required_vars:
    if not os.getenv(var):
        print(f"❌ Missing {var}")
    else:
        print(f"✅ {var} loaded")
        
print(f"\n🔗 Endpoint: {os.getenv('AZURE_OPENAI_ENDPOINT')}")
print(f"🤖 Model: {os.getenv('AZURE_OPENAI_DEPLOYMENT_NAME')}")

✅ AZURE_OPENAI_ENDPOINT loaded
✅ AZURE_OPENAI_API_KEY loaded
✅ AZURE_OPENAI_DEPLOYMENT_NAME loaded

🔗 Endpoint: https://chwestbr-foundry-projec-resource.openai.azure.com/
🤖 Model: gpt-4o


## 2. LangChain Setup

In [31]:
from langchain_openai import AzureChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain

# Initialize Azure OpenAI through LangChain
# Note: AzureChatOpenAI uses the chat completion API (not completion API)
# This ensures we get proper conversation context and system message support
llm = AzureChatOpenAI(
    azure_endpoint=os.getenv('AZURE_OPENAI_ENDPOINT'),
    api_key=os.getenv('AZURE_OPENAI_API_KEY'),
    api_version=os.getenv('AZURE_OPENAI_API_VERSION', '2024-02-15-preview'),
    deployment_name=os.getenv('AZURE_OPENAI_DEPLOYMENT_NAME'),
    temperature=0.7,
    max_tokens=500
)

print("🦜 LangChain ChatOpenAI initialized (using chat completion API)")

🦜 LangChain ChatOpenAI initialized (using chat completion API)


## 3. Basic Chat

In [32]:
# Simple message invocation
messages = [
    SystemMessage(content="You are a helpful AI assistant."),
    HumanMessage(content="Hello! Can you tell me a joke?")
]

response = llm.invoke(messages)
print("🤖 AI Response:")
print(response.content)

🤖 AI Response:
Of course! Here's one for you:

Why don't skeletons fight each other?

Because they don't have the guts! 😄


## 4. Prompt Templates

In [33]:
# Create a prompt template
prompt_template = ChatPromptTemplate.from_messages([
    ("system", "You are an expert {role}. Provide helpful advice in a {tone} tone."),
    ("human", "{question}")
])

# Use the template
prompt = prompt_template.format_messages(
    role="software developer",
    tone="friendly and encouraging", 
    question="What's the best way to learn Python?"
)

response = llm.invoke(prompt)
print("🎯 Template Response:")
print(response.content)

🎯 Template Response:
Learning Python is an exciting journey, and you're going to have a lot of fun with it! Python is beginner-friendly and very versatile, so you're already on the right track by choosing it. Here’s a roadmap to help you get started and stay motivated:

---

### **1. Start with the Basics**
- **Install Python**: Download and install the latest version of Python from [python.org](https://www.python.org/).
- **Learn Syntax and Fundamentals**: Start with basic concepts like variables, data types, loops, conditionals, and functions. There are many free resources like [w3schools](https://www.w3schools.com/python/) and [Python’s official tutorial](https://docs.python.org/3/tutorial/).
- **Interactive Platforms**: Use beginner-friendly platforms like [Codecademy](https://www.codecademy.com/learn/learn-python-3), [freeCodeCamp](https://www.freecodecamp.org/), or [Real Python](https://realpython.com/).

---

### **2. Practice, Practice, Practice**
- **Write Code Daily**: Set as

## 5. Conversation Chain with Memory

> **📝 Note on Jupyter Output:** Due to a known Jupyter quirk with async operations, this example is split across multiple cells to avoid output duplication. The pattern is: Setup → Execute → Display. This separation also makes the code flow clearer by separating concerns.

In [34]:
# Create conversation with automatic memory - the modern LangChain way!
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# Create a simple prompt template that includes message history
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant."),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}")
])

# Create the basic chain
chain = prompt | llm

# Create message history store (in memory for this demo)
store = {}

def get_session_history(session_id: str) -> ChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

# Wrap the chain with message history - LangChain handles everything automatically!
conversation = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history",
)

print("💭 Conversation setup complete! Ready to chat with memory...")

💭 Conversation setup complete! Ready to chat with memory...


In [35]:
# Demo conversation with automatic memory
session_id = "demo_session"

# First exchange - just chat naturally
response1 = conversation.invoke(
    {"input": "My name is Alice and I love Python programming."}, 
    config={"configurable": {"session_id": session_id}}
)

# Second exchange - LangChain automatically remembers previous messages
response2 = conversation.invoke(
    {"input": "What's my name and what do I love?"}, 
    config={"configurable": {"session_id": session_id}}
)

In [36]:
# Display the conversation results
print("🗣️ Conversation Results:\n")

print("👤 User: My name is Alice and I love Python programming.")
print(f"🤖 AI: {response1.content}\n")

print("👤 User: What's my name and what do I love?")
print(f"🤖 AI: {response2.content}\n")

# Show the conversation history that LangChain automatically maintained
history = get_session_history(session_id)
print(f"📚 LangChain automatically stored {len(history.messages)} messages!")
for msg in history.messages:
    msg_type = "👤 User" if msg.type == "human" else "🤖 AI"
    print(f"{msg_type}: {msg.content[:60]}{'...' if len(msg.content) > 60 else ''}")

🗣️ Conversation Results:

👤 User: My name is Alice and I love Python programming.
🤖 AI: Hi, Alice! It's great to meet you. Python programming is awesome—whether you're building websites, analyzing data, or automating tasks, Python can do it all. What kinds of projects are you working on, or what do you enjoy most about Python?

👤 User: What's my name and what do I love?
🤖 AI: Your name is **Alice**, and you love **Python programming**! 😊

📚 LangChain automatically stored 4 messages!
👤 User: My name is Alice and I love Python programming.
🤖 AI: Hi, Alice! It's great to meet you. Python programming is awe...
👤 User: What's my name and what do I love?
🤖 AI: Your name is **Alice**, and you love **Python programming**!...


## 6. Chaining: Simple and Advanced Examples

This section showcases LangChain's core strength - **chaining operations** to create multi-step workflows. We'll start with simple chaining and then show a more complex content creation pipeline.

In [37]:
# SIMPLE CHAINING EXAMPLE
from langchain_core.output_parsers import StrOutputParser

# Create a simple two-step chain: Analyze → Summarize
analyze_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an analyst. Analyze the given topic and provide 3 key insights."),
    ("human", "Topic: {topic}")
])

summarize_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a writer. Take these insights and write a brief, engaging summary."),
    ("human", "Insights: {insights}")
])

# Build individual chains
analyze_chain = analyze_prompt | llm | StrOutputParser()
summarize_chain = summarize_prompt | llm | StrOutputParser()

print("🔗 Simple chains created!")
print("   • Analyze chain: Takes topic → Returns insights")  
print("   • Summarize chain: Takes insights → Returns summary")
print("\n⚡ Next: We'll chain them together!")

🔗 Simple chains created!
   • Analyze chain: Takes topic → Returns insights
   • Summarize chain: Takes insights → Returns summary

⚡ Next: We'll chain them together!


In [40]:
# SIMPLE CHAINING - No data transformation needed!
print("Creating simple chain: Analyze -> Summarize")

# The problem: Our chains expect different input keys
# analyze_chain expects: {"topic": "..."}
# summarize_chain expects: {"insights": "..."}

# LangChain solution: Use RunnablePassthrough to map outputs to inputs
from langchain_core.runnables import RunnablePassthrough

# Create a simple chain that pipes the output of one to the input of the next
simple_chain = (
    analyze_chain
    | (lambda insights: {"insights": insights})  # Just map the string to the expected key
    | summarize_chain
)

# Execute the simple chain
topic = "remote work productivity"

print(f"Executing simple chain for '{topic}'...")
print("This will: Analyze topic -> Generate summary\n")

result = simple_chain.invoke({"topic": topic})

print("Simple chain complete!")
print(f"Final summary: {result[:100]}...")

print("\nThis demonstrates LangChain's core value:")
print("   • Automatic data flow between steps")
print("   • Single invoke() call for multi-step workflow")
print("   • Clean, declarative pipeline definition")

Creating simple chain: Analyze -> Summarize
Executing simple chain for 'remote work productivity'...
This will: Analyze topic -> Generate summary

Simple chain complete!
Final summary: Remote work offers employees the flexibility to tailor their schedules and environments for maximum ...

This demonstrates LangChain's core value:
   • Automatic data flow between steps
   • Single invoke() call for multi-step workflow
   • Clean, declarative pipeline definition
Simple chain complete!
Final summary: Remote work offers employees the flexibility to tailor their schedules and environments for maximum ...

This demonstrates LangChain's core value:
   • Automatic data flow between steps
   • Single invoke() call for multi-step workflow
   • Clean, declarative pipeline definition


In [41]:
# ADVANCED CHAINING - Content Creation Pipeline
print("\n" + "="*60)
print("🚀 ADVANCED EXAMPLE: Content Creation Pipeline")
print("="*60)

# More sophisticated chains for content creation
research_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a research expert. Generate 5 bullet points about the topic."),
    ("human", "Research topic: {topic}")
])

draft_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a content writer. Write a short blog post based on these research points."),
    ("human", "Research: {research}")
])

review_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an editor. Review this draft and suggest 3 improvements."),
    ("human", "Draft to review: {draft}")
])

# Build the chains
research_chain = research_prompt | llm | StrOutputParser()
draft_chain = draft_prompt | llm | StrOutputParser()  
review_chain = review_prompt | llm | StrOutputParser()

# Create the full content pipeline: Research → Draft → Review
content_pipeline = (
    research_chain
    | (lambda research: {"research": research})
    | draft_chain
    | (lambda draft: {"draft": draft})
    | review_chain
)

print("📝 Content pipeline created: Research → Draft → Review")

# Execute the advanced pipeline
topic = "benefits of remote work"

print(f"\n🎯 Executing content pipeline for '{topic}'...")
print("⚡ Single call will: Research → Write Draft → Generate Review\n")

result = content_pipeline.invoke({"topic": topic})

print("✅ Content pipeline complete!")
print(f"📋 Editor review: {result[:120]}...")

print(f"\n💡 Advanced pipeline benefits:")
print("   • 3-step workflow in one call")
print("   • Automatic context passing")
print("   • Complex content creation made simple")
print("   • Each step builds on the previous")


🚀 ADVANCED EXAMPLE: Content Creation Pipeline
📝 Content pipeline created: Research → Draft → Review

🎯 Executing content pipeline for 'benefits of remote work'...
⚡ Single call will: Research → Write Draft → Generate Review

✅ Content pipeline complete!
📋 Editor review: This draft effectively highlights the benefits of remote work, but there are areas where clarity, depth, and engagement ...

💡 Advanced pipeline benefits:
   • 3-step workflow in one call
   • Automatic context passing
   • Complex content creation made simple
   • Each step builds on the previous
✅ Content pipeline complete!
📋 Editor review: This draft effectively highlights the benefits of remote work, but there are areas where clarity, depth, and engagement ...

💡 Advanced pipeline benefits:
   • 3-step workflow in one call
   • Automatic context passing
   • Complex content creation made simple
   • Each step builds on the previous


In [42]:
# Show what chaining accomplishes vs manual orchestration
print("=" * 70)
print("🔗 LANGCHAIN CHAINING vs MANUAL ORCHESTRATION")
print("=" * 70)

print("\n❌ MANUAL APPROACH (what we avoided):")
print("   # Research step")
print("   research_result = research_chain.invoke({'topic': topic})")
print("   # Draft step") 
print("   draft_result = draft_chain.invoke({'research': research_result})")
print("   # Review step")
print("   final_result = review_chain.invoke({'draft': draft_result})")
print("   # Multiple calls, manual data passing, error handling")

print("\n✅ LANGCHAIN CHAINING:")
print("   # Single call does everything!")
print("   result = content_pipeline.invoke({'topic': topic})")
print("   # Automatic data flow, built-in error handling")

print(f"\n📊 PIPELINE RESULT:")
print("-" * 50)
print(result[:200] + "..." if len(result) > 200 else result)

print(f"\n🎯 KEY BENEFITS OF CHAINING:")
print("   • Single invoke() call for multi-step workflows")
print("   • Automatic data flow between pipeline steps") 
print("   • Built-in error handling and retries")
print("   • Clean, readable pipeline definitions")
print("   • Reusable and composable components")

print(f"\n💡 COMPARE TO RAW OPENAI SDK:")
print("   • 3 separate API calls to manage")
print("   • Manual data transformation between steps")  
print("   • Custom error handling for each step")
print("   • Context passing and state management")
print("   • Retry logic implementation")

print(f"\n✨ SIMPLE YET POWERFUL:")
print("   • Basic chaining: analyze_chain | summarize_chain")
print("   • Advanced pipeline: research → draft → review")
print("   • Same pattern scales from 2 steps to 20+ steps")

print("\n" + "=" * 70)
print("🚀 This is why LangChain excels at AI workflows!")
print("=" * 70)

🔗 LANGCHAIN CHAINING vs MANUAL ORCHESTRATION

❌ MANUAL APPROACH (what we avoided):
   # Research step
   research_result = research_chain.invoke({'topic': topic})
   # Draft step
   draft_result = draft_chain.invoke({'research': research_result})
   # Review step
   final_result = review_chain.invoke({'draft': draft_result})
   # Multiple calls, manual data passing, error handling

✅ LANGCHAIN CHAINING:
   # Single call does everything!
   result = content_pipeline.invoke({'topic': topic})
   # Automatic data flow, built-in error handling

📊 PIPELINE RESULT:
--------------------------------------------------
This draft effectively highlights the benefits of remote work, but there are areas where clarity, depth, and engagement can be improved. Here are three suggested improvements:

---

### 1. **Enhance t...

🎯 KEY BENEFITS OF CHAINING:
   • Single invoke() call for multi-step workflows
   • Automatic data flow between pipeline steps
   • Built-in error handling and retries
   • Clean,

## 7. Summary

This notebook demonstrated LangChain's core concepts and advanced capabilities:

✅ **LangChain Setup**: Initialized AzureChatOpenAI with environment configuration (uses chat completion API)

✅ **Basic Chat**: Simple message invocation with system and human messages

✅ **Prompt Templates**: Reusable prompt structures with variable substitution

✅ **Modern Memory**: Automatic conversation management using `RunnableWithMessageHistory`

✅ **Advanced Chaining**: Multi-step content creation pipeline (Research → Draft → Review → Polish)

### Key LangChain Benefits:
- **Chat Completion API**: Uses modern chat completion endpoints (not legacy completion)
- **Abstraction**: Higher-level interface than raw OpenAI SDK
- **Automatic Memory**: Built-in conversation history with `RunnableWithMessageHistory`
- **Templates**: Reusable prompt structures with variable substitution
- **Chain Orchestration**: Seamlessly coordinate multi-step AI workflows
- **Session Management**: Easy conversation sessions with automatic message tracking

### Why LangChain Shines:
The **content creation pipeline** shows LangChain's real value - what would require complex manual orchestration (managing 4 separate API calls, passing context between steps, handling errors) becomes a simple, declarative workflow. Try building that same pipeline with raw OpenAI SDK calls!

### Modern LangChain Patterns:
- **RunnableWithMessageHistory**: Current standard for conversation memory
- **MessagesPlaceholder**: Clean way to inject conversation history
- **Chain Composition**: Use pipe operators for readable multi-step workflows
- **Automatic Context Passing**: Data flows seamlessly between pipeline steps
- **Error Handling**: Built-in resilience for complex workflows

### Next Steps:
- Explore LangChain agents for tool-calling workflows
- Try different memory types and stores (Redis, SQL, etc.)
- Experiment with conditional chains and branching logic
- Integrate with vector databases for RAG applications

This notebook focused on essential LangChain patterns, from basic chat to advanced multi-step orchestration - showing why LangChain excels at complex AI workflows.